In [4]:
def get_fin(stock):
    import requests
    from bs4 import BeautifulSoup
    from selenium import webdriver
    from selenium.webdriver.common.keys import Keys
    import pandas as pd
    import time

    url = f"https://www.finnomena.com/stock/{stock}"
    driver = webdriver.Chrome('./chromedriver') 
    driver.get(url) 

    time.sleep(2) 
    html = driver.page_source
    soup = BeautifulSoup(html, "html.parser")

    driver.close()

    all_divs = soup.find_all('div', {'class' : "content" })
    n = 0
    quarter = []
    total_asset = []
    total_debt = []
    shareholder_equity = []
    paid_up_capital = []
    total_revenue = []
    net_profit = []
    esp = []
    roa = []
    roe = []
    net_profit_margin = []
    market_capitalization = []
    p_e = []
    p_bv = []
    dividend_yield = []

    buffer = []
    n_data = 0
    for i in all_divs:
        if n == 0:
            year = i.find_all('div', {'class' : "year" })
            for j in year:
                q = j.text[2:]+j.text[1:2]+j.text[:1]
                quarter.append(q)
            quarter[-1] = quarter[-1][-1:]+quarter[-1][-2:-1]+quarter[-1][:-2]
            n+=1
            n_data = len(quarter)
            continue
        elif n == 1:
            each = i.find_all('div', {'class' : "data-each" })
            for j in each:
                if j.text == 'N/A':
                    data = "null"
                elif j.text == '':
                    data = "null"
                elif j.text != '':
                    data = float(j.text.replace(',',''))
                if len(total_asset) < n_data:
                    total_asset.append(data)
                elif len(total_debt) < n_data:
                    total_debt.append(data)
                elif len(shareholder_equity) < n_data:
                    shareholder_equity.append(data)
                elif len(paid_up_capital) < n_data:
                    paid_up_capital.append(data)
                elif len(total_revenue) < n_data:
                    total_revenue.append(data)
                elif len(buffer) < 2*n_data:
                    buffer.append('1')
                elif len(net_profit) < n_data:
                    net_profit.append(data)
                elif len(buffer) < (2+4)*n_data:
                    buffer.append('1')
                elif len(esp) < n_data:
                    esp.append(data)
        elif n == 2:
            each = i.find_all('div', {'class' : "data-each" })
            for j in each:
                if j.text == 'N/A':
                    data = "null"
                elif j.text == '':
                    data = "null"
                elif j.text != '':
                    data = float(j.text.replace(',',''))
                if len(roa) < n_data:
                    roa.append(data)
                elif len(roe) < n_data:
                    roe.append(data)
                elif len(buffer) < (2+4+2)*n_data:
                    buffer.append('1')
                elif len(net_profit_margin) < n_data:
                    net_profit_margin.append(data)
        elif n == 3:
            each = i.find_all('div', {'class' : "data-each" })
            for j in each:
                if j.text == 'N/A':
                    data = "null"
                elif j.text == '':
                    data = "null"
                elif j.text != '':
                    data = float(j.text.replace(',',''))
                if len(buffer) < (2+4+2+1)*n_data:
                    buffer.append("1")
                elif len(market_capitalization) < n_data:
                    market_capitalization.append(data)
                elif len(p_e) < n_data:
                    p_e.append(data)
                elif len(p_bv) < n_data:
                    p_bv.append(data)
                elif len(buffer) < (2+4+2+1+1)*n_data:
                    buffer.append("1")
                elif len(dividend_yield) < n_data:
                    dividend_yield.append(data)
        n+=1

    stock_statement = {"quarter":quarter,"total_asset":total_asset,"total_debt":total_debt,"shareholder_equity":shareholder_equity,"paid_up_capital":paid_up_capital,"total_revenue":total_revenue,"net_profit":net_profit,"esp":esp,"roa":roa,"roe":roe,"net_profit_margin":net_profit_margin,"market_capitalization":market_capitalization,"p_e":p_e,"p_bv":p_bv,"dividend_yield":dividend_yield}
    df = pd.DataFrame(stock_statement)
    print(df)
    row_lists = df.apply(lambda row: row.tolist(), axis=1)

    return row_lists.to_list() 


In [2]:
def insert_financial(stock_id,finance):
    from sqlite3 import connect
    conn = connect('stock.db',timeout=10)#connect to database
    cursor = conn.cursor()
    cursor.execute(f"INSERT INTO new_financial_statement VALUES (null,{stock_id},'{finance[0]}',{finance[1]},{finance[2]},{finance[3]},{finance[4]},{finance[5]},{finance[6]},{finance[7]},{finance[8]},{finance[9]},{finance[10]},{finance[11]},{finance[12]},{finance[13]},{finance[14]})")
    conn.commit()#commit change to db
    conn.close()#disconnect

In [6]:
stock = "GFPT"
get_fin(stock)

C:\Users\Krittapon\AppData\Local\Temp\ipykernel_20636\1796920636.py:10: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('./chromedriver')


      quarter total_asset total_debt shareholder_equity paid_up_capital  \
0      2556Q1    12849.08    6804.61            6044.48         1253.82   
1      2556Q2    12396.93    6050.26            6346.67         1253.82   
2      2556Q3    12851.65    5891.48            6960.17         1253.82   
3      2556Q4    13103.15    5645.39            7457.76         1253.82   
4      2557Q1    13626.18    5823.02            7803.16         1253.82   
5      2557Q2    13759.43    5911.67            7847.76         1253.82   
6      2557Q3    13568.87    5210.38            8358.49         1253.82   
7      2557Q4    13737.41    5004.43            8732.98         1253.82   
8      2558Q1    14330.97    5375.72            8955.25         1253.82   
9      2558Q2    14388.01    5730.44            8657.57         1253.82   
10     2558Q3    14530.54     5542.7            8987.83         1253.82   
11     2558Q4    14976.32    5533.07            9443.25         1253.82   
12     2559Q1    14926.94

[['2556Q1',
  12849.08,
  6804.61,
  6044.48,
  1253.82,
  3938.76,
  130.48,
  0.1,
  1.01,
  2.24,
  3.31,
  10657.48,
  261.02,
  1.92,
  0.0],
 ['2556Q2',
  12396.93,
  6050.26,
  6346.67,
  1253.82,
  4174.56,
  304.93,
  0.24,
  2.42,
  4.92,
  7.3,
  10218.64,
  55.46,
  1.71,
  0.0],
 ['2556Q3',
  12851.65,
  5891.48,
  6960.17,
  1253.82,
  4423.26,
  615.76,
  0.49,
  4.88,
  9.25,
  13.92,
  11284.39,
  22.44,
  1.8,
  0.0],
 ['2556Q4',
  13103.15,
  5645.39,
  7457.76,
  1253.82,
  4474.11,
  466.63,
  0.37,
  3.6,
  6.47,
  10.43,
  16048.91,
  14.12,
  2.33,
  0.0],
 ['2557Q1',
  13626.18,
  5823.02,
  7803.16,
  1253.82,
  4069.84,
  427.03,
  0.34,
  3.2,
  5.6,
  10.49,
  18180.4,
  12.09,
  2.47,
  2.41],
 ['2557Q2',
  13759.43,
  5911.67,
  7847.76,
  1253.82,
  4554.35,
  485.84,
  0.38,
  3.55,
  6.21,
  10.67,
  16048.91,
  8.92,
  2.08,
  2.73],
 ['2557Q3',
  13568.87,
  5210.38,
  8358.49,
  1253.82,
  4886.02,
  510.72,
  0.41,
  3.74,
  6.3,
  10.45,
  25327.1

In [3]:
from data.function import Stock,Categories
# stock = Categories().get_all_stock()
stock = [""]
for i in stock:
    try:
        temp = Stock(i)
        id = temp.get_stock_id()
        financial = get_fin(i)
        for j in financial:
            insert_financial(id,j)
    except:
        pass

C:\Users\Krittapon\AppData\Local\Temp\ipykernel_20636\448694861.py:10: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('./chromedriver')
C:\Users\Krittapon\AppData\Local\Temp\ipykernel_20636\448694861.py:10: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('./chromedriver')
C:\Users\Krittapon\AppData\Local\Temp\ipykernel_20636\448694861.py:10: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('./chromedriver')
C:\Users\Krittapon\AppData\Local\Temp\ipykernel_20636\448694861.py:10: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('./chromedriver')
C:\Users\Krittapon\AppData\Local\Temp\ipykernel_20636\448694861.py:10: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdr